<a href="https://colab.research.google.com/github/Sathiyapramod/TensorFlow_image_processing/blob/main/CatVsDog_image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import tensorflow as tf
from tensorflow import keras
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [24]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [25]:

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [26]:
fid = drive.ListFile({'q':"title='dataset.zip'"}).GetList()[0]['id']
f=drive.CreateFile({'id':fid})
f.GetContentFile('dataset.zip')

In [27]:
!unzip dataset.zip

Archive:  dataset.zip
replace dataset/test_set/cats/cat.4001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [28]:
train_data = ImageDataGenerator(rescale=1./255)

TARGET_SIZE=(128,128)

In [29]:
train_generator = train_data.flow_from_directory(
    '/content/dataset/training_set',
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


In [30]:
val_data = ImageDataGenerator(rescale=1./255)

In [31]:
val_generator = val_data.flow_from_directory(
    '/content/dataset/test_set',
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


In [42]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(128,128,3)),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(40,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [45]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [46]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5214 - loss: 1.6320

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 60ms/step - accuracy: 0.5214 - loss: 1.6298 - val_accuracy: 0.5715 - val_loss: 0.6908
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.5716 - loss: 0.7692 - val_accuracy: 0.5550 - val_loss: 0.6837
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.5518 - loss: 0.7081 - val_accuracy: 0.5975 - val_loss: 0.6659
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.5845 - loss: 0.6778 - val_accuracy: 0.5245 - val_loss: 0.6957
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.6020 - loss: 0.6594 - val_accuracy: 0.6230 - val_loss: 0.6521
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.5982 - loss: 0.6644 - val_accuracy: 0.6120 - val_loss: 0.6646
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 58ms/step - accuracy: 0.6178 - loss: 0.6500 - val_accuracy: 0.6185 - val_loss: 0.6545
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.6234 - loss: 0.6467 - val_accurac

In [50]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

TARGET_SIZE=(128,128)


uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/'+fn
  img = image.load_img(path,target_size=TARGET_SIZE)
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  images = np.vstack([x])
  classes = model.predict(images,batch_size=1)
  print(classes[0])

  if classes[0]>0.5:
    print(fn + "is a Dog")
  else:
    print(fn + "is a Cat")

Saving demo_cat.jpg to demo_cat (1).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
[2.2984548e-08]
demo_cat (1).jpgis a Cat
